In [1]:
import sys
print(sys.executable)

c:\projetos\FD-TD\.venv\Scripts\python.exe


In [5]:
!pip3 install opencv-python --break-system-packages
!pip3 install pyqt5 --break-system-packages
!pip3 install scipy --break-system-packages

In [1]:
# import pycuda.driver as cuda
# import pycuda.autoinit
# from pycuda.compiler import SourceModule
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import cv2
import time

from config import *
from util import *
from leapfrog2D import *

ENGINE = 'qt5'

if ENGINE == 'qt5':
    %matplotlib qt5
else:
    %matplotlib widget

In [ ]:
#S = configSimulation('laser pulse', anecroic=True)
#S = configSimulation('single lens')
#S = configSimulation('single lens plane')
#S = configSimulation('yagi_antenna')
#S = configSimulation('antireflex')
#S = configSimulation('opticalfiber')
#S = configSimulation('optical ckt')
#S = configSimulation('dual optcal fiber')
#S = configSimulation('refraction')  # mega demo!

# homework simulations
#S = configSimulation('wall right')
#S = configSimulation('wall left')
#S = configSimulation('wall above')
#S = configSimulation('wall below')
#S = configSimulation('room')
#S = configSimulation('closed room')
#S = configSimulation('room with lens')
#S = configSimulation('room with receiver')
S = configSimulation('flat room with receiver')

# config variables
Nt = S['Nt']
dt = S['dt']
Nx = S['Nx']
Ny = S['Ny']

# --- 1: Preparação da Mensagem ---

def string_to_bits(s: str) -> list[int]:
    """Converte string para lista de bits (8 bits por char)"""
    bits = []
    for char in s:
        bin_str = f"{ord(char):08b}"
        bits.extend(int(b) for b in bin_str)
    return bits

# 1. Definição da mensagem
MENSAGEM = "Oi"  
bits_msg = string_to_bits(MENSAGEM)

print(f"Mensagem: '{MENSAGEM}'")
print(f"Bits a transmitir: {bits_msg}")
print(f"Total de bits: {len(bits_msg)}")

# 2. Configura a duração de cada bit na simulação
# Nt é o número total de passos de tempo que a simulação tem (ex: 1500)
# Precisamos garantir que a mensagem cabe no tempo da simulação.
steps_per_bit = int(Nt / (len(bits_msg) + 2)) 

# (+2 é uma folga para o sinal não acabar exatamente no último segundo)

print(f"Duração de cada bit: {steps_per_bit} steps")
if steps_per_bit < 20:
    print("ALERTA: O tempo de simulação (Nt) é muito curto para essa mensagem! Aumente S['Nt'] na config.")

# ---------------------------------------

# --- CONFIGURAÇÃO DA TRANSMISSÃO ---
MODULATE_ASK = True   # <--- Mude para False não modula o sinal(sem mensagem)

MENSAGEM = "Oi"
bits_msg = string_to_bits(MENSAGEM)

# Recalcula steps apenas se a modulação estiver ativa, senão usa 1 (pra não dar erro de divisão por zero)
if MODULATE_ASK:
    # --- CALCULO AUTOMÁTICO DO TEMPO ---
    
    # Define quantos passos de tempo dura CADA BIT. 
    # 120 é um bom número para garantir que a onda senoidal tenha ciclos suficientes (frequência 40)
    steps_per_bit = 120 
    
    # Tempo extra para a onda viajar da fonte até o receptor (Propagation Delay)
    buffer_propagacao = 1000 
    
    # Calcula o novo Nt necessário
    total_steps_necessarios = (len(bits_msg) * steps_per_bit) + buffer_propagacao
    
    # Se o tempo padrão da config for curto, aumentamos ele
    if total_steps_necessarios > Nt:
        print(f"⚠️ AVISO: Aumentando Nt de {Nt} para {total_steps_necessarios} para caber a mensagem.")
        Nt = total_steps_necessarios
    
    print(f"MODO ASK: Enviando '{MENSAGEM}' ({len(bits_msg)} bits).")
    print(f"Duração por bit: {steps_per_bit} steps.")
    print(f"Tempo Total Simulação: {Nt} steps.")
else:
    steps_per_bit = 1 
    print("MODO NORMAL: Sinal contínuo (sem modulação de bits).")

# initial field values
Hx = np.zeros((S['Ny'],S['Nx']))
Hy = np.zeros((S['Ny'],S['Nx']))
Ez = np.zeros((S['Ny'],S['Nx']))

# medium
s = S['s']
er = S['er']
ur = S['ur']
wire = S['wire']
dt = S['dt']

# sources
sources_parameters = S['sources_parameters']
sources_names = S['sources_names']

# FD-TD constants (normalization discretization steps, etc)
[c1, c2, c3, c4] = leapFrog2D_constants(S['er'], S['ur'], S['s'], S['dt']);

parameters = {}
parameters['c1'] = c1;
parameters['c2'] = c2;
parameters['c3'] = c3;
parameters['c4'] = c4;
parameters['dx'] = S['dx']
parameters['dy'] = S['dy']
parameters['wire'] = wire

# colormaps
[M1, M2, M3, M4] = customColormaps();
Imaterial = composeImage(s, wire, er, M2, M3, M4)

# Drawing stuff

fig, ax = plt.subplots(figsize=(10, 8))
ax_img = ax.imshow(Imaterial)
ax.set_aspect('equal')
ax.set_axis_off()

fourcc = cv2.VideoWriter_fourcc(*"X264")
video_out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (Nx,Ny))

# ---------------------------------------------------------
# CONFIGURAÇÃO DO RECEIVER 
# ---------------------------------------------------------
receiver_data = [] # Lista para guardar o sinal capturado
rx_pixel = 0
ry_pixel = 0

has_receiver = 'receivers' in S and len(S['receivers']) > 0

if has_receiver:
    # Pega o primeiro receiver da lista
    rec = S['receivers'][0] 
    
    # Converte de 0.0-1.0 para Pixels (Inteiros)
    rx_pixel = int(rec['cx'] * Nx)
    ry_pixel = int(rec['cy'] * Ny)
    
    # Garante que está dentro da tela
    rx_pixel = np.clip(rx_pixel, 0, Nx-1)
    ry_pixel = np.clip(ry_pixel, 0, Ny-1)
    
    print(f"Receiver posicionado em: X={rx_pixel}, Y={ry_pixel}")

# ---------------------------------------------------------

# main time loop 
t = 0
n = 1

# Cria uma string única com todos os bits para exibir na tela (Ex: "01001111...")
bits_str = "".join(str(b) for b in bits_msg)

# Guarda a amplitude original configurada no config.py para usar caso MODULATE_ASK seja False
amplitude_original = sources_parameters[0]['amplitude']

while n < Nt:
    n = n + 1;
    t = t + dt

    # --- LÓGICA DE MODULAÇÃO (ASK) ---
    if MODULATE_ASK:
        # Descobre qual bit transmitir agora
        bit_index = int(n / steps_per_bit)

        if bit_index < len(bits_msg):
            bit_atual = bits_msg[bit_index]
            
            if bit_atual == 1:
                # BIT 1: Amplitude Máxima (usa o valor original configurado)
                sources_parameters[0]['amplitude'] = amplitude_original
            else:
                # BIT 0: Silêncio
                sources_parameters[0]['amplitude'] = 0
        else:
            # Acabou a mensagem, silêncio total
            sources_parameters[0]['amplitude'] = 0
            
    # Se MODULATE_ASK for False, ele nem entra no if e usa a amplitude original continuamente
    # -------------------------------------------------
    # inject sources
    for source_parameter, source_name in zip(sources_parameters, sources_names):
        source_parameter['time'] = t;
        Ez += generateSource(Nx, Ny, source_name,source_parameter)
    
    
    # Perform Mathemagic
    leapFrog2D(Ez, Hy, Hx, parameters)

    # -----------------------------------------------------
    # CAPTURA DO SINAL PELO RECEIVER
    # -----------------------------------------------------
    
    if has_receiver:
        # Salva o valor do Campo Elétrico (Ez) naquele ponto exato
        val = Ez[ry_pixel, rx_pixel]
        receiver_data.append(val)
    # -----------------------------------------------------
    # draw stuff
    if n%5==0:
        I = addEzToComposite(Imaterial, Ez/5.0, M1)
        # -------------------------------------------------
        # DESENHAR O 'X' DO RECEIVER NA IMAGEM
        # -------------------------------------------------
        if has_receiver:
            
            # Tamanho do marcador
            size = 5 
            
            # Cor vermelha [255, 0, 0]
            
            y_min = max(0, ry_pixel - size)
            y_max = min(Ny, ry_pixel + size)
            x_min = max(0, rx_pixel - size)
            x_max = min(Nx, rx_pixel + size)

            # Desenha um quadrado verde onde está o receiver para vê-lo
            I[y_min:y_max, x_min:x_max, 0] = 0  # R
            I[y_min:y_max, x_min:x_max, 1] = 255  # G
            I[y_min:y_max, x_min:x_max, 2] = 0  # B

        # --- DESENHA O TEXTO DA MENSAGEM ---
        # Escreve a Mensagem Original na Imagem usando OpenCV
        cv2.putText(
            I, 
            f"Msg: {MENSAGEM}", (20, 30), # Posição (x, y)
            cv2.FONT_HERSHEY_SIMPLEX, # Fonte
            0.8,                     # Tamanho da fonte
            (255, 255, 255),         # Cor (Branco)
            2                        # Espessura
        )

        # B. Escreve os Bits (Ex: "Bits: 01001...")
        # Desenhando duas vezes: uma cinza (base) e uma verde (bit atual)
        # Texto base (todos os bits em cinza claro)
        cv2.putText(
            I, 
            f"Bits: {bits_str}", 
            (20, 60), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.6, 
            (180, 180, 180), # Cinza
            1
        )

        # C. Destaca o Bit Atual (Verde)
        if MODULATE_ASK:
            # Calcula qual índice estamos transmitindo agora
            idx_atual = int(n / steps_per_bit)
            
            if idx_atual < len(bits_str):
                # Truque visual: Desenhamos apenas o bit atual por cima, na posição certa.
                # Como calcular a posição exata do caractere é chato no OpenCV, 
                # vamos destacar escrevendo o bit atual grandão ao lado da mensagem.
                
                bit_val = bits_str[idx_atual]
                cor_bit = (0, 255, 0) if bit_val == '1' else (0, 0, 255) # Verde se 1, Azul se 0
                
                # Indicador de progresso (Círculo ou Texto ao lado)
                cv2.putText(
                    I, 
                    f"TX: {bit_val}",        # Mostra o bit sendo enviado agora
                    (250, 30),               # Ao lado da mensagem "Oi"
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8, 
                    cor_bit, 
                    2
                )
                
                # (Opcional) Barra de progresso visual simples abaixo dos bits
                start_x = 80 # Margem onde começam os bits visualmente
                progresso_x = start_x + (idx_atual * 12) # 12 pixels por char aprox
                cv2.line(I, (start_x, 70), (progresso_x, 70), (0, 255, 0), 2)


        if ENGINE == 'qt5':
            ax_img.set_data(I)
            plt.pause(0.001)
        else:
            ax.clear()
            ax.imshow(I)
            clear_output(wait=True)
            display(fig)

if n%2 == 0:
    video_out.write(I[:,:,[2, 1, 0]])


[configSimulation] Unknown configuration name: flat room with receiver
Mensagem: 'Oi'
Bits a transmitir: [0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1]
Total de bits: 16
Duração de cada bit: 83 steps
⚠️ AVISO: Aumentando Nt de 1500 para 2920 para caber a mensagem.
MODO ASK: Enviando 'Oi' (16 bits).
Duração por bit: 120 steps.
Tempo Total Simulação: 2920 steps.
Receiver posicionado em: X=560, Y=208


KeyboardInterrupt: 

In [ ]:
video_out.release()

In [7]:
plt.imshow(S['s'])
plt.show()

In [23]:
# Plota o sinal capturado
if has_receiver:
    plt.figure(figsize=(10,4))
    plt.plot(receiver_data)
    plt.title(f"Sinal recebido na posição X={rx_pixel}, Y={ry_pixel}")
    plt.xlabel("Tempo (steps)")
    plt.ylabel("Amplitude (Ez)")
    plt.grid(True)
    plt.show()
else:
    print("Nenhum receiver configurado.")